In [3]:
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import mandelbrot_matrix

reload(mandelbrot_matrix)

plt.style.use('seaborn')

re_lim=(-2,1) 
im_lim=(-1,1)

In [38]:
def generate_latinHyperCube():


    width_re = re_lim[1] - re_lim[0]
    width_im = im_lim[1] - im_lim[0]

    #print(width_re)
    #print(width_im)

    re_all = []
    im_all = []

    n = 1000
    factor_re = width_re/n
    factor_im = width_im/n

    low_re = re_lim[0]
    high_re = low_re + factor_re


    low_im = im_lim[0]
    high_im = low_im + factor_im

    for i in range(n):
        re = np.random.uniform(low_re, high_re) # type: ignore
        im = np.random.uniform(low_im,high_im) # type: ignore
        low_re = high_re
        high_re = low_re + factor_re
        low_im = high_im
        high_im = low_im + factor_im
        re_all.append(re)
        im_all.append(im)
    
    print(im_all)
    start = 0
    while(start+1 < n):
        selectIndex = np.random.randint(start,n)
        #swap start and selectIndex
        temp = im_all[selectIndex]
        im_all[selectIndex] = im_all[start]
        im_all[start] = temp
        start = start+1

    re_all = np.array(re_all)
    im_all = np.array(im_all)
    #print(im_all.shape)
    random_points = re_all+im_all*1j 
    
    return random_points

    #plt.plot(re_all, im_all, "o")    
    #plt.show()


$ \bar{X}_{j+1} = \bar{X}_{j} + \frac{X_{j+1} - \bar{X}_{j}}{j+1} $

$ {S}_{j+1}^{2} = (1-\frac{1}{j})S_{j}^{2}+(j+1)(\bar{X}_{j+1} - \bar{X}_{j})^{2} $

In [40]:
%%time

n_sim = 100    # number of simulations to run
n_size = 1000   # number of ramdom points
threshold = 1000 # threshold for mandelbrot
data = []

for sim in range(n_sim):
    
    random_points = generate_latinHyperCube()
    
    # create the counter matrix "n" and the z matrix
    n = np.zeros(n_size)
    z = np.zeros(n_size)

    MAX_ITER = threshold
    # iterate until the threshold is reached
    for k in range(MAX_ITER):
        # add one to n_ij only if abs(z_ij[k])<=2
        n = np.where(np.abs(z) <= 2, n+1, n)
        # calculate z_ij[k+1] only if abs(z_ij[k])<=2
        z = np.where(np.abs(z) <= 2, np.square(z) + random_points, z)
                
    #print(z)
    count_inside = np.size(np.where(n>=threshold))
    #print(count_inside)
    
    # monte carlo
    total_points = n_size
    avg = count_inside/total_points
    
    
    
    data.append(avg)
    #print(avg)
    # average will follow a normal distribution for large values of n i.e. sim
    # alpha = 0.5
    # alpha/2 = 0.25. Z(alpha/2) = 1.96
    # percent confidence interval  = 100*(1-alpha) = 100*(1-0.5) = 95%
    # stop when 2*z*s/(sqrt(sim)) < l. 
    # if we just allow 1% error then 1% of 1.506484 = 0.01506484 ~ 0.015. Therefore our l is 0.015/Area of rectangle
    # l = (0.015)/6 = 0.0025
    # sample mean of (j+1)
    # Our samples - averages
    

prev_estimated_mean = np.sum(data)/(n_sim)
print(prev_estimated_mean)

mean_difference = 0

for i in range(n_sim):
    mean_difference += (data[i]-prev_estimated_mean)**2
    
prev_estimated_variance = mean_difference/(n_sim-1)



l = 0.0025
z = 1.96
estimated_std_deviation = np.sqrt(prev_estimated_variance)
acceptable_window = 2*z*estimated_std_deviation/np.sqrt(n_sim)
#print(acceptable_window)


while(acceptable_window > l):
    
        re, im, n = mandelbrot_matrix.random_mandelbrot_points(n_size, re_lim, im_lim,threshold=threshold)
        
        count_inside = np.size(np.where(n>=threshold))
    
        # monte carlo
        total_points = n_size
        avg = count_inside/total_points
        #data.append(avg)
        
        
        
        new_estimated_mean = prev_estimated_mean + (avg - prev_estimated_mean)/(n_sim+1)
        new_estimated_variance = (1-1/(n_sim))*prev_estimated_variance + (n_sim+1)*(new_estimated_mean-prev_estimated_mean)**2
        
        
        n_sim +=1
        #print(n_sim)
        prev_estimated_variance = new_estimated_variance
        prev_estimated_mean = new_estimated_mean
        
        estimated_std_deviation = np.sqrt(new_estimated_variance)
        acceptable_window = 2*z*estimated_std_deviation/np.sqrt(n_sim)
        #print(acceptable_window)




[-0.9985915916513601, -0.9962829157287206, -0.9946551455195141, -0.9936636819832155, -0.991859675375857, -0.9880072084233084, -0.9867977206481463, -0.9843211818687461, -0.9822302458896992, -0.9817567292795866, -0.9795735973554941, -0.9762246027363973, -0.9757175816990199, -0.9732110953664864, -0.9706308622925107, -0.9687017858883948, -0.9667509871031277, -0.964935139057289, -0.9628120574669355, -0.9610207767474999, -0.959856458315647, -0.9574054121390635, -0.9557113412025565, -0.9538447731201487, -0.9500730749787163, -0.9492973895490535, -0.9474980214559353, -0.944566144386274, -0.9426082472688716, -0.9405572255442031, -0.9392026274808817, -0.9370128462080501, -0.9347047516363968, -0.9321507528553142, -0.9305652412322728, -0.9291609265486361, -0.9271917526975706, -0.9255170974481438, -0.9236215622981488, -0.9213250163678646, -0.918511566748421, -0.9169458462989292, -0.9151855669061983, -0.9129453571597236, -0.9101530740657584, -0.9081421524163629, -0.9078760630766827, -0.90404970609823

[-0.9981502052456841, -0.9973845106083592, -0.9945415057798151, -0.9939901260421237, -0.9909065510514087, -0.9883046208940005, -0.9872758950111623, -0.9851427736976942, -0.9826365104632905, -0.9809019514516499, -0.9788580639975257, -0.9778899535608798, -0.9747220340977113, -0.9725180210911591, -0.9703400657023797, -0.9695701543590536, -0.9666641353854036, -0.9658560565168448, -0.9620999650183086, -0.9609462001623547, -0.9587689803875113, -0.9575442364731075, -0.9551118856818117, -0.9531588789912309, -0.9512796638167128, -0.9493228778937043, -0.9461969448712192, -0.9457571306086462, -0.942776353906532, -0.9403874824900383, -0.9394467141301782, -0.9376955524244523, -0.9350618030020905, -0.9321673833196009, -0.9300787242740789, -0.9287364687196474, -0.9260432855903128, -0.9240012205940832, -0.9235691701398868, -0.920902699962483, -0.9196845302180083, -0.9178819590355795, -0.91475529971176, -0.9122984503147451, -0.9100273792566831, -0.9091254900239091, -0.9071678927614475, -0.9040604325526

[-0.99925157823924, -0.9974437109603614, -0.9958665322520129, -0.9928349586590609, -0.990931823532294, -0.9897535138920606, -0.986986490276509, -0.9851264740374021, -0.9825544669745895, -0.9808436483278505, -0.9798320929167685, -0.9762858223474796, -0.975503783976336, -0.9734199708821192, -0.9706401523438478, -0.9681593016663824, -0.966512872683032, -0.9656450991898435, -0.9637250218074245, -0.9611591288829547, -0.958651829194023, -0.9562345342551546, -0.9543759123649136, -0.9522056650181854, -0.9501649464733058, -0.9490252188418354, -0.9462681728557133, -0.9456533443165261, -0.9423547378782465, -0.9413058673917544, -0.9390007122521442, -0.9362180123439251, -0.935795177554144, -0.9338590413131169, -0.9319590030973002, -0.9299735549570944, -0.9272470270616542, -0.9242556576791684, -0.9228008894921402, -0.9219943193099198, -0.9183388227796309, -0.9171542641213712, -0.915298656953915, -0.9128090130906676, -0.9112258361858676, -0.9091201300679518, -0.9069132680758595, -0.9044315890604087, 

[-0.9984611688651225, -0.9976101583832871, -0.9955978097632433, -0.993934251887165, -0.9900123204659049, -0.9885628949974945, -0.9877345614950347, -0.984815320082372, -0.9836406433248256, -0.981384481514897, -0.9787225711180539, -0.9763065656257274, -0.9745253954784017, -0.9727174175053299, -0.9708315420958361, -0.9686694482827093, -0.9661310987520385, -0.9659471017514364, -0.9639841599442223, -0.9609344724650669, -0.9583323206440961, -0.9576539377389157, -0.9559752941494608, -0.9536375166866115, -0.9509318694515704, -0.9492126992433403, -0.9472298462362407, -0.9455874656844259, -0.9426183055599529, -0.9412531056258593, -0.939970978920263, -0.937910351482294, -0.9350469360611581, -0.932511516987893, -0.9308223453320482, -0.9293282408846703, -0.9274148738168096, -0.9245384756269779, -0.922484910344923, -0.9208086355427386, -0.9185708604658617, -0.9176250774605166, -0.9141766110288488, -0.912026822627703, -0.9117905629047146, -0.908835691167988, -0.9078330230068103, -0.9053031387627373, 

[-0.9982268789270781, -0.9965387058635735, -0.9944913544913879, -0.9924627940085058, -0.9910535343460576, -0.9890888043100492, -0.9870468183956902, -0.9850611556275816, -0.9829836969767455, -0.9814591339334313, -0.978017692216046, -0.9772131401405452, -0.974482987931384, -0.9737548723518147, -0.9707022654819388, -0.9680837949948827, -0.9662355108929379, -0.9649955349358796, -0.9632551236608944, -0.9607489897225489, -0.95907369372003, -0.9571557629499435, -0.9551625598051677, -0.9535790160108459, -0.950411529729255, -0.9492302980165874, -0.9467858265122508, -0.945852018180697, -0.9428043647252187, -0.9409974920278176, -0.9397411423413785, -0.9374667550840358, -0.9346663143370987, -0.9321073217408228, -0.9313884005248638, -0.9292297158502701, -0.9266435603474271, -0.9253554844589572, -0.9228220378515136, -0.9216857331296282, -0.9197544107474974, -0.9165683257337786, -0.9142965619350175, -0.9128699999536999, -0.9116922430033075, -0.9091793955009779, -0.9064948926685996, -0.904183345737887

...

[-0.9981275345565833, -0.9960300739509845, -0.9955199652156316, -0.9926171100931064, -0.9909878690556225, -0.9899495797415109, -0.9872978829364515, -0.9859614404204834, -0.9821285843831588, -0.9800978885326239, -0.979313706106338, -0.9774635251303586, -0.9756049275708621, -0.9728990748763042, -0.9705015725178158, -0.9688647359991968, -0.9679183579075995, -0.9657257267099485, -0.9621951168484719, -0.9618450018524652, -0.959118517773008, -0.9569626995451865, -0.9557583589104546, -0.9537963540603773, -0.9501083826332707, -0.9499126713825554, -0.9475251191415787, -0.9452424740296865, -0.94320276549504, -0.9403066909804657, -0.9381008118939804, -0.9370892693028671, -0.9354675294340548, -0.933794600648108, -0.9319516690435888, -0.9295670963077215, -0.9279862284835104, -0.9255184518433113, -0.9227814086634931, -0.9206078189997867, -0.9181640308911521, -0.9160629669790186, -0.9158605041222103, -0.913000860023369, -0.9110546877188496, -0.9085528469821468, -0.9068347794371119, -0.904186456342171

[-0.9981740488547608, -0.9960416104122515, -0.9949853662441844, -0.9938523062834477, -0.9914785780797207, -0.9884151610111626, -0.9879717254151357, -0.9840554847428157, -0.9822320325648813, -0.9815519615222624, -0.9793725412000616, -0.9773534031245296, -0.9743363289872873, -0.9733992772018648, -0.9708370091888422, -0.9691655778779935, -0.966290713277312, -0.9655873974734952, -0.9639690137822711, -0.961319026388934, -0.9589724437066943, -0.9565537953724892, -0.9547306593458185, -0.9533755548211001, -0.9506504931057207, -0.9498314034479594, -0.9462468452063649, -0.9444780271765384, -0.9434529425800964, -0.9407143602533009, -0.9388643028761058, -0.9373944821704742, -0.9351892744994053, -0.9328488238723727, -0.9310876157593527, -0.9298946020328033, -0.9273906995396665, -0.9255216404512652, -0.9231016891580038, -0.9217106441257444, -0.9193313294949664, -0.9172380694765138, -0.9146457528004888, -0.912211045365137, -0.9116118093293035, -0.9096044796860407, -0.9077059172719946, -0.904452460525

[-0.9989520015446748, -0.9976235143368622, -0.9943577992194069, -0.9935749685660014, -0.9910767594292849, -0.9890023473774924, -0.9868352290239365, -0.9854574808246749, -0.9830437208063847, -0.9815380312296212, -0.9791899576476467, -0.9777649365540598, -0.975958839593834, -0.9735470428091024, -0.9704689942674811, -0.9687987778885087, -0.9678973144074364, -0.9648593894013957, -0.9621112647917095, -0.961482239414445, -0.9580739703175684, -0.9574971935633618, -0.9541748666231792, -0.9522484448065753, -0.9511212009207648, -0.9485365856303355, -0.9474602297025251, -0.9441498642425985, -0.9424190570280364, -0.9404334639193643, -0.9389214435238163, -0.9376954967264928, -0.9359606686780064, -0.9323426478205653, -0.9316764932801718, -0.9298768177434349, -0.9260806337549959, -0.9248548111220508, -0.9226519751877912, -0.9207346386126951, -0.9195009262126858, -0.9179583014923789, -0.9153244975372227, -0.913606148984937, -0.9115875032571409, -0.9083927583518616, -0.9060997692603233, -0.905094932385

[-0.9998230883927332, -0.9964577662228105, -0.9947090930753225, -0.9921486090131015, -0.9902000303590044, -0.9885939934248027, -0.9863850526905705, -0.9848061367883751, -0.9822232642267105, -0.9805394166362656, -0.9791247809097956, -0.9761828400133116, -0.9758250609973397, -0.9733628209240375, -0.971709866808713, -0.9695021089207158, -0.9672032099052464, -0.9643064034661334, -0.9636281559438328, -0.9606952551083259, -0.9596084693371959, -0.9572667020558479, -0.9544591275850869, -0.9522931518403489, -0.9505725882859392, -0.9485592464572582, -0.9479611747420911, -0.9455153210138639, -0.943170033066912, -0.9410109702767687, -0.9395334213009909, -0.9369386726615283, -0.9358708838293176, -0.9326898729134038, -0.9312434215265457, -0.9282745959362861, -0.9277545381518465, -0.9256400092910032, -0.9224106845340971, -0.9203056615053286, -0.919496116006531, -0.9177602060289368, -0.9146137410105334, -0.9137500989734646, -0.9117133314656511, -0.9090529399334157, -0.9065247837043517, -0.904601690712

CPU times: user 6.01 s, sys: 31.1 ms, total: 6.04 s
Wall time: 6.04 s


In [41]:
print(n_sim)

# less simulations required as compared to pure random sampling

328
